In [3]:
seed = 666

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from time import time

import re, random

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import spacy
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
import pandas as pd

In [4]:
corpus_df = pd.read_csv('corpus_df.csv').drop(columns=['Unnamed: 0'])
corpus_df.head()
# corpus_df.shape # (1693, 3)

,lyrics,artist,genre
0,I know a girl who thinks it goes she'll make y...,flaming lips,indie
1,"All those bugs buzzin' round your head Well, t...",flaming lips,indie
2,"Something in you, it jitters like a moth And I...",flaming lips,indie
3,Their wasn't any snow on Christmas eve and I k...,flaming lips,indie
4,"You can walk among us, but you can't walk on b...",flaming lips,indie


In [ ]:
# corpus_df['genre'].value_counts()
# punk     1097
# indie     596

In [5]:
# corpus_df['lyrics_lemma'] = corpus_df["lyrics"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
# corpus_df.head()

In [6]:
def lemmatize_df(df):
    
    """Function that takes in a DF. Must have a column called ['lyrics']
       Returns lemmatized version of that data."""
    
    df = df.copy()
    
    df['lyrics'] = df['lyrics'].apply(lambda s: s.lower())
    
    return df

In [7]:
corpus_df.head()

,lyrics,artist,genre
0,I know a girl who thinks it goes she'll make y...,flaming lips,indie
1,"All those bugs buzzin' round your head Well, t...",flaming lips,indie
2,"Something in you, it jitters like a moth And I...",flaming lips,indie
3,Their wasn't any snow on Christmas eve and I k...,flaming lips,indie
4,"You can walk among us, but you can't walk on b...",flaming lips,indie


In [8]:
new_user_input = 'HELLO I LOVE MUSIC'

In [9]:
import pandas as pd
user_input_df = pd.DataFrame([new_user_input], columns=['lyrics'])

In [10]:
step_1 = lemmatize_df(user_input_df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus_df['lyrics_lemma'], corpus_df['genre'], test_size=.25, random_state=seed,
                                                    stratify=corpus_df['genre'])
# train = pd.concat([X_train, y_train], axis=1)
# test = pd.concat([X_test, y_test], axis=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# train['genre'].value_counts()
# punk     822
# indie    447

# test['genre'].value_counts()
# punk     275
# indie    149

In [ ]:
# def inspect(vectoriser, X):
#     # Fit and transform
#     start = time()
#     print(f"There are {vectoriser.fit_transform(X).shape[1]} columns.\n")
#     end = time()
#     print(f"Took {round((end-start),2)} seconds.\n")
    
#     # Inspect tokens
#     tokens = list(vectoriser.vocabulary_.keys())
#     tokens.sort()
#     print(f"Example tokens: {tokens[:50]}\n")
    
#     # Inspect ignored tokens
#     ignored = vectoriser.stop_words_
#     if len(ignored)==0:
#         print("No token is ignored.")
#     elif len(ignored)>50:
#         print(f"Example ignored tokens: {random.sample(ignored, 50)}")
#     else:
#         print(f"Example ignored tokens: {ignored}")

In [ ]:
vectoriser = TfidfVectorizer(token_pattern=r'[a-z]+', stop_words='english', min_df=10, max_df=.97)
#inspect(vectoriser, X_train)

In [ ]:
X_train_pr = vectoriser.fit_transform(X_train)
X_train_pr

In [ ]:
pd.DataFrame(X_train_pr.todense(), columns=vectoriser.get_feature_names())

In [ ]:
m = RandomForestClassifier(random_state=seed)
m.fit(X_train_pr, y_train)
m.score(X_train_pr, y_train)

In [ ]:
X_test_pr = vectoriser.transform(X_test)
pd.DataFrame(X_test_pr.todense(), columns=vectoriser.get_feature_names())

In [ ]:
m.score(X_test_pr, y_test)

In [ ]:
m.predict_proba(['some lyrics'])

In [12]:
from sklearn.preprocessing import FunctionTransformer

In [14]:
lemmatizer = FunctionTransformer(lemmatize_df)

In [15]:
lemmatizer.fit_transform(user_input_df)

,lyrics
0,hello i love music


In [16]:
text_pipeline = make_pipeline(lemmatizer,
                              TfidfVectorizer(token_pattern=r'[a-z]+', stop_words='english', min_df=10, max_df=.97),
                              RandomForestClassifier(max_depth=5, random_state=seed))
# text_pipeline.fit(X_train, y_train)

In [17]:
text_pipeline

Pipeline(memory=None,
         steps=[('functiontransformer',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function lemmatize_df at 0x7ff7765289d0>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, validate=False)),
                ('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8'...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=5, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decre